# ✍️ AuraDB — OpenAI RAG (Ejercicio de **consultas**) con LangChain
Cuaderno de **alumno** para consultar un grafo **ya creado** en *Neo4j AuraDB* (IMDB).  
Usaremos **LangChain + OpenAI** para NL→Cypher y responder en español.

**Estructura de cada sección:**  
- Explicación + **Pistas clave**  
- Celda de **código con TODOs** (tú la completas)  
- **### Solución (mostrar/ocultar)** y una celda con la **solución ejecutable**

## ▶️ Instalación rápida
**Docs útiles**
- AuraDB (conectar apps): https://neo4j.com/docs/aura/connecting-applications/overview/  
- Neo4j Python Driver: https://neo4j.com/docs/python-manual/current/  
- LangChain (Neo4j): https://python.langchain.com/docs/integrations/graphs/neo4j  
- Prompt templates: https://python.langchain.com/docs/guides/prompt_templates/

In [ ]:
# Recomendado: ejecutar esta celda primero (reinicia kernel si actualiza mucho)
%pip install -q python-dotenv neo4j langchain langchain-community langchain-openai tiktoken

## 1) Configuración inicial (API Keys + modelo + conexión Neo4j)
**Objetivo:** preparar imports, cargar `.env`, configurar **OpenAI** y crear el conector `Neo4jGraph` (solo lectura).

**Pistas clave**
- `from dotenv import load_dotenv`; `load_dotenv()`  
- Variables: `OPENAI_API_KEY`, `OPENAI_MODEL` (`gpt-4.1-mini`)  
- `from langchain_openai import ChatOpenAI` + `os.environ["OPENAI_API_KEY"] = ...`  
- `from langchain_community.graphs import Neo4jGraph` y `Neo4jGraph(url=..., username=..., password=...)`  
- Prueba: `graph.query("RETURN 1 AS ok")`

In [ ]:
# TODO (Config inicial)
# 1) Imports y .env
# import os
# from dotenv import load_dotenv
# load_dotenv()

# 2) Lee variables (o pon valores directos para pruebas)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<tu-openai-key>"
# OPENAI_MODEL   = os.getenv("OPENAI_MODEL")   or "gpt-4.1-mini"
# NEO4J_URI      = os.getenv("NEO4J_URI")      or "neo4j+s://<tu-host>.databases.neo4j.io"
# NEO4J_USERNAME = os.getenv("NEO4J_USERNAME") or "neo4j"
# NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "<tu-contraseña>"

# 3) OpenAI (LangChain)
# from langchain_openai import ChatOpenAI
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# 4) Conexión a AuraDB (solo lectura)
# from langchain_community.graphs import Neo4jGraph

# graph.query("RETURN 1 AS ok")

## 2) Esquema del grafo (solo lectura)
**Objetivo:** refrescar e inspeccionar el esquema para guiar la generación de Cypher.

**Pistas clave**
- `graph.refresh_schema()`  
- `print(graph.schema[:1000])`

In [ ]:
# TODO (Esquema)
# graph.refresh_schema()
# print(graph.schema[:1000] + ("..." if len(graph.schema) > 1000 else ""))

## 3) Prompt de generación de Cypher (solo lectura)
**Objetivo:** construir un `PromptTemplate` que **prohíba escrituras** y use `{schema}` y `{question}`.

**Pistas clave**
- `from langchain.prompts import PromptTemplate`  
- `input_variables=["schema","question"]`  
- Incluir: “NO usar CREATE/MERGE/SET/DELETE/REMOVE/DROP/LOAD CSV”

In [ ]:
# TODO (Prompt Cypher)
# from langchain.prompts import PromptTemplate
# Usa las variables "schema" y "question"

## 4) Utilidad `clean_fences(text)`
**Objetivo:** si el LLM devuelve ```cypher ...```, quitar fences y etiquetas para quedarnos solo con el Cypher.

**Pistas clave**
- `text.strip()` y `splitlines()`  

In [ ]:
# TODO (clean_fences)
# def clean_fences(text: str) -> str:
#     # 1) strip
#     # 2) si empieza por ``` quitar fences
#     # 3) si la primera línea no empieza por MATCH/CALL/RETURN/WITH, saltarla
#     # 4) unir líneas
#     # return text_limpio
#     ...

## 5) Validador: **solo lectura**
**Objetivo:** rechazar Cypher con escrituras (CREATE/MERGE/SET/DELETE/REMOVE/DROP/LOAD CSV / CALL dbms...).

**Pistas clave**
- `import re` + `re.compile(r"...", re.IGNORECASE)`  
- Función `is_read_only(cy: str) -> bool` que busca con `.search(...)`
- Si empieza por ```, quitar backticks y primera línea si no empieza por `MATCH/CALL/RETURN/WITH`

In [ ]:
# TODO (solo-lectura)
# import re
# READ_ONLY_PATTERN = re.compile(r"...", re.IGNORECASE)  # palabras prohibidas
# def is_read_only(cy: str) -> bool:
#     # True si NO hay match
#     # return not bool( ... )
#     ...

## 6) Función `generate_cypher(llm, schema, question)`
**Objetivo:** construir el prompt, invocar el LLM, limpiar fences y validar que sea **solo lectura**.

**Pistas clave**
- `prompt = CYTHER_PROMPT_TMPL.format(schema=schema, question=question)`  
- `text = llm.invoke(prompt).content`  
- `cy = clean_fences(text)` y `is_read_only(cy)`

In [ ]:
# TODO (generate_cypher)
# def generate_cypher(llm, schema: str, question: str) -> str:
#     # prompt = ...
#     # text   = ...
#     # cy     = ...
#     # if not is_read_only(cy): raise ValueError(...)
#     # return cy
#     ...

## 7) Función `run_cypher(graph, cypher)`
**Objetivo:** ejecutar la consulta de solo lectura y devolver registros (lista de dicts).

**Pistas clave**
- `graph.query(cypher, params or {})`

In [ ]:
# TODO (run_cypher)
# def run_cypher(graph, cypher: str, params=None):
#     # return graph.query(...)
#     ...

## 8) Función `answer_question(question)`
**Objetivo:** generar Cypher, **imprimirlo**, ejecutarlo y resumir con el LLM en español.

**Pistas clave**
- `cy = generate_cypher(llm, graph.schema, question)`  
- `print(cy)` antes de ejecutar  
- `rows = run_cypher(graph, cy)`  
- `PromptTemplate` para redactar respuesta corta a partir de filas

In [ ]:
# TODO (answer_question)

# ANSWER_PROMPT_TMPL = (
#     input_variables=["question","rows"],
#     template=(
#     )
# )
# def answer_question(question: str, max_rows: int = 20, show_cypher: bool = True):
#     # cy = ...
#     # if show_cypher: print("— Cypher generado —"); print(cy); print("———————")
#     # rows = ...
#     # short_rows = rows[:max_rows]
#     # ans = llm.invoke(ANSWER_PROMPT_TMPL.format(question=question, rows=short_rows)).content
#     # return {"cypher": cy, "rows": rows, "answer": ans}
#     ...

### Solución (mostrar/ocultar)

## 9) Pruebas guiadas
Ejecuta estas pruebas cuando termines los ejercicios.

In [ ]:
#  — Prueba NL→Cypher


In [ ]:
#  — Prueba Cypher directo


## 10) Troubleshooting
- `AuthenticationError`: revisa `NEO4J_URI`, `NEO4J_USERNAME`, `NEO4J_PASSWORD` (usa `neo4j+s://` con Aura).  
- `ServiceUnavailable`: problemas de red/URI o instancia parada.  
- **El Cypher no se ve**: aquí se imprime antes de ejecutar (`answer_question(..., show_cypher=True)`).  
- **LLM propone escritura**: tu validador la detecta y lanza error.